In [1]:
%matplotlib inline

# 中文分詞(Pytorch)

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import pylab
import PIL  
import os
import pickle
import codecs
import glob
import math
import random
import builtins
import string
import numpy as np
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
def str_full_to_half(in_str):
    out_str = []
    for char in in_str:
        inside_code = ord(char)
        if inside_code == 0x3000 or inside_code == 12288 or char==string.whitespace: # 全形空格直接轉換
             out_str.append(' ')
        elif inside_code >= 65281 and inside_code <= 65374:
            inside_code -= 0xfee0
            out_str.append(chr(inside_code))
        else:
            out_str.append(char)
        
    return ''.join(out_str)

In [4]:
as_train=codecs.open('../Data/ex12_train/as_training.utf8',encoding='utf-8-sig').read()
cityu_train=codecs.open('../Data/ex12_train/cityu_training.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_train=as_train.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_train=cityu_train.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data=as_train+'\r\n'+cityu_train #把兩個語料合併
data=data.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data=str_full_to_half(data) #把所有全形轉半形

raw_data_train=data.split('\r\n')#分行

raw_data_train=[row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_train] #移除分行字元

print(raw_data_train[:20])

train_data_len=[len(txt.split('|'))  for txt in  raw_data_train]
print(max(train_data_len))



['時間|:', '三月|十日|(|星期四|)|上午|十時|。', '地點|:', '學術|活動|中心|一樓|簡報室|。', '主講|:', '民族所|所長|莊英章|先生|。', '講題|:', '閩|、|台|漢人|社會|研究|的|若干|考察|。', '李|院長|於|二月|二十六日|至|三月|十五日|赴|美|訪問|,', '期間|將|與|在|美|院士|商討|院務|,', '與|美國|大學|聯繫|商討|長期|合作|事宜|,', '並|辦理|加州|大學|退休|等|手續|。', '出國|期間|院務|由|羅|副院長|代行|。', '總辦事處|秘書組|主任|戴政|先生|請辭|獲准|,', '所|遺|職務|自|三月|一日|起|由|近代史|研究所|研究員|陶英惠|先生|兼任|。', '植物|研究所|所長|周昌弘|先生|當選|第三世界|科學院|(|The|Third|World|Academy|of|Sciences|,', '簡稱|TWAS|)|院士|。', 'TWAS|係|一九八三年|由|Prof|Adbus|Salam|(|巴基斯坦籍|,', '曾|獲|諾貝爾獎|)|發起|成立|,', '會員|遍佈|63|個|國家|,']
211


In [5]:
as_test=codecs.open('../Data/ex12_train/as_testing_gold.utf8',encoding='utf-8-sig').read()
cityu_test=codecs.open('../Data/ex12_train/cityu_test_gold.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_test=as_test.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_test=cityu_test.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data_test=as_test+'\r\n'+cityu_test #把兩個語料合併
data_test=data_test.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data_test=str_full_to_half(data_test) #把所有全形轉半形

raw_data_test=data_test.split('\r\n')#分行

raw_data_test=[row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_test] #移除分行字元

print(raw_data_test[:20])
test_data_len=[len(txt.split('|'))  for txt in  raw_data_test]
print(max(test_data_len))

['許多|社區|長青|學苑|多|開設|有|書法|、|插花|、|土風舞班|,', '文山區|長青|學苑|則|有|個|十分|特別|的|「|英文|歌唱班|」|,', '成員|年齡|均|超過|六十|歲|,', '這|群|白髮蒼蒼|,', '爺爺|、|奶奶級|的|學員|唱起|英文|歌|來|字正腔圓|,', '有模有樣|。', '對|他們|來說|,', '能|與|一|群|志同道合|的|朋友|共同|回味|年少|時期|流行|的|歌曲|,', '才|是|參加|英文|歌唱班|最|大|樂趣|。', '長青|學苑|英文|歌唱班|昨日|在|社教館|舉行|「|英文|懷念|金曲|演唱會|」|,', '曲目|包括|「|大江東去|」|、|「|月河|」|、|「|真善美|」|等|大眾|耳熟能詳|的|英文|歌曲|。', '難得|公開|演唱|,', '這些|有|著|豐富|人生|閱歷|的|學員|絲毫|不|覺得|緊張|怯場|,', '只|見|台|上|唱|得|盡興|,', '台|下|不少|聽眾|也|一時|技癢|跟|著|唱和|起來|。', '長青|學苑|英文|歌唱班|成立|至今|已|兩|年|,', '目前|成員|約|廿五|人|,', '年齡|都|在|六十|歲|以上|,', '其中|以|軍公教|退休|人員|居多|,', '並|有|現任|大學|教授|,']
104


In [6]:
vocabs=sorted(set(list(''.join(raw_data_train))))
vocabs.remove('|')
vocabs.insert(0,'/Unknow')
vocabs.insert(0,'<PAD>')

print(len(vocabs))
print(vocabs[:100])

6296
['<PAD>', '/Unknow', ' ', '!', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '°', '·', '×', 'ˊ', 'ˋ', '˙', 'Α', 'Β', 'Γ', 'Ε', 'Η']


In [7]:
char_to_index=dict((w, i) for i, w in enumerate(vocabs))
index_to_char=dict((i, w) for i, w in enumerate(vocabs))

print({key:char_to_index[key] for key in list(char_to_index.keys())[:100]})

{'凸': 560, '聘': 4279, '夜': 1208, '嫩': 1348, 'H': 39, '隉': 5789, '驥': 6050, '鷺': 6202, '健': 434, '訪': 4965, '酒': 5504, '胥': 4328, '過': 5413, '邱': 5464, '晝': 2378, '戶': 1980, '鄰': 5493, '筑': 3943, '吐': 760, '荼': 4546, '謝': 5061, '欞': 2710, '勇': 633, '臺': 4416, '噌': 994, '偏': 430, '些': 257, '噸': 1014, '綸': 4120, '繆': 4164, '娉': 1298, '呲': 798, '盾': 3641, '逑': 5381, '槐': 2642, '悌': 1826, '捉': 2085, '羅': 4211, '充': 499, '酬': 5513, '民': 2784, '嬤': 1361, '馨': 6003, '邐': 5453, '伺': 317, '粳': 4029, '毽': 2780, '煽': 3213, '密': 1428, '嚴': 1029, '滁': 3018, '衩': 4852, '卜': 690, '換': 2143, '鍬': 5650, '仄': 275, '壹': 1197, '橫': 2677, '斂': 2299, '膨': 4389, '真': 3650, '遵': 5435, '迓': 5354, '冊': 529, 'Ⅰ': 117, '鴃': 6144, '佶': 350, '銅': 5591, '礽': 3793, '斯': 2317, '咆': 810, '轍': 5332, '訶': 4969, '猥': 3325, '題': 5926, '殞': 2751, '憲': 1927, '察': 1436, '袞': 4867, '地': 1074, '凰': 556, '冇': 527, '麥': 6224, '段': 2758, '揹': 2154, '殼': 2761, '●': 137, '榫': 2630, '托': 1995, '勾': 652, '薯': 4685, '廛': 1670, '槌': 2639

In [8]:
idx_train=0
idxs_train=np.arange(len(raw_data_train))
np.random.shuffle(idxs_train)

idx_test=0
idxs_test=np.arange(len(raw_data_test))
np.random.shuffle(idxs_test)

def get_next_minibatch(minibatch_size,is_train=True):
    global idx_train,idxs_train,raw_data_train,idx_test,idxs_test,raw_data_test

    groundtruths=[]
    idx=idx_train
    idxs=idxs_train
    raw_data=raw_data_train
    if is_train==False:
        idx=idx_test
        idxs=idxs_test
        raw_data=raw_data_test
    np.random.shuffle(raw_data)
    #定義輸出向量形狀
    feature_arr=np.zeros((minibatch_size,128))
    label_arr=np.zeros((minibatch_size,128))
    length_arr=np.zeros((minibatch_size))
    batch=0
    while batch<minibatch_size:
        seq_len=len(raw_data[idxs[idx]].replace('|',''))
        if seq_len<=128 and seq_len>0:
            length_arr[batch]=seq_len
            
            groundtruth_seq=[]
            words=raw_data[idxs[idx]].split('|') #轉換成分詞後的詞清單

            pos=0
            #BMES=>[0,1,2,3]
            for word in words:
                for i in range(len(word)):
                    #如果在字典中則取出其索引
                    if word[i] in char_to_index:
                        feature_arr[batch,pos]=char_to_index[word[i]]
                    #否則定為未知
                    else:
                        feature_arr[batch,pos]=char_to_index['/Unknow']

                    #轉換為BMES
                    if len(word)==1 and i==0: #S 自己就是一個單詞
                        label_arr[batch,pos]=3
                    elif i==0: #B 是一個詞的開始
                        label_arr[batch,pos]=0
                    elif i==len(word)-1:  #E 是一個詞的結束
                        label_arr[batch,pos]=2
                    else: #M 是一個詞的中間
                        label_arr[batch,pos]=1  
                    pos+=1

                groundtruth_seq.append(word)
            groundtruths.append(groundtruth_seq)
            batch+=1
        idx+=1
        if idx>len(idxs):
            idx=0
            np.random.shuffle(idxs)
    idx_train=idx
    if is_train==False:
        idx_test=idx
    return feature_arr.astype(np.int64),label_arr.astype(np.int64),length_arr.astype(np.int64),groundtruths
        
x_feature,y_label,x_len,ground_truths=get_next_minibatch(2)
print(x_feature.shape)
print(y_label.shape)
print(x_len)
print(ground_truths)

(2, 128)
(2, 128)
[30 11]
[['進入', '西堤島', '大都', '為', '了', '觀賞', '聖母院', '及', 'La', 'Conciergerie', ','], ['要', '化解', '臺灣', '的', '教育', '問題', ',']]


In [9]:

class CharRNN(nn.Module):
    def __init__(self, batch_size,vocab_size,embed_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.batch_size=batch_size
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embed_size = embed_size
        self.embedding = nn.Embedding(vocab_size, embed_size,padding_idx=0)
        self.lstm = nn.LSTM(
            input_size=self.embed_size,
            hidden_size=self.hidden_size,
            num_layers=2,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.5)
        self.liner = nn.Linear(hidden_size*2, output_size) 
    def init_hidden(self):
        #2*2的意思是指:lstm層數*雙向lstm
        hidden_a = torch.randn(2*2, self.batch_size, self.hidden_size).to(device)
        hidden_b = torch.randn(2*2, self.batch_size, self.hidden_size).to(device)
        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)
        return (hidden_a, hidden_b)
    def forward(self,x, x_lengths):
        #Lstmt初始的隱藏狀態
        self.hidden = self.init_hidden()
        
        #print(x.shape)
        output = self.embedding(x)
        #print(output.shape)#(批次，序列長度,嵌入層)
        
        output = torch.nn.utils.rnn.pack_padded_sequence(output, x_lengths, batch_first=True)
        
        output, self.hidden = self.lstm(output, self.hidden)
        output, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        #print(output.shape)#(批次，序列長度,隱藏層*2)
        
        output = output.contiguous()
        output = output.view(-1, output.shape[2]) 
        #print(output.shape)#(批次*序列長度,隱藏層*2)
        
        output = self.dropout(output)
        
        
        output = self.liner(output)
        #print(output.shape)#(批次*序列長度,隱藏層*2)
        output = F.softmax(output, dim=1)
        output = output.view(self.batch_size, output.size(0)//self.batch_size, self.output_size)
        #print(output.shape)#(批次,序列長度,輸出)
        return output


In [ ]:
def masked_loss(pred_y, target_y,input_x,num_class=5):
        # flatten all predictions
        pred_y = pred_y.view(-1,num_class)
        #print('pred_y'+str(pred_y.shape))
        # flatten all the labels
        target_y = target_y.view(-1)[:pred_y.shape[0]]
        #print('target_y'+str(target_y.shape))
        input_x = input_x.view(-1)[:pred_y.shape[0]]
        #print('input_x'+str(input_x.shape))
        
        

        # create a mask by filtering out all tokens that ARE NOT the padding token
        mask = (input_x>0).float()
        #print('mask'+str(mask.shape))
        # count how many tokens we have
        nb_tokens = int(torch.sum(mask).data.item())
        # pick the values for the label and zero out the rest with the mask
        pred_y = (1-pred_y[range(pred_y.shape[0]),target_y]) * mask

        # compute cross entropy loss which ignores all <PAD> tokens
        ce_loss = torch.sum(pred_y) / nb_tokens
        return ce_loss
    


In [ ]:
learning_rate=0.0005
minibatch_size=64
num_epochs=20

#宣告模型結構
#output_size還是4，不要計算pad
model = CharRNN(batch_size=minibatch_size,vocab_size=len(vocabs),embed_size=256, hidden_size=256, output_size=4)
model.to(device)
if os.path.exists('Models/word_segment_pytorch.lstm'):
    model=torch.load('Models/word_segment_pytorch.lstm')
    print('recovered!!')

model.train()
model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.95, 0.999))
criterion = nn.NLLLoss()



epochs=30
accuracy_numerator=[]
accuracy_denominator=[]
print('start epoch!')
for epoch in range(epochs):
    mbs=0
    while mbs<2000:
        features, labels,lens,ground_truths=get_next_minibatch(minibatch_size)
        inputs, targets,seq_len = torch.from_numpy(features.astype(np.int64)), torch.from_numpy(labels.astype(np.int64)), torch.from_numpy(lens.astype(np.int64))

        inputs, targets,seq_len = Variable(inputs), Variable(targets), Variable(seq_len).to(device)
        
        #所有輸入的sequence必須由長到短排序才可以輸入
        seq_len_sorted, sorted_idx = seq_len.sort(descending=True)
        inputs_sorted, targets_sorted,seq_len_sorted=inputs[sorted_idx].to(device), targets[sorted_idx].to(device),seq_len_sorted.to(device)
        
        model.zero_grad()
        
        
        output= model.forward(inputs_sorted,seq_len_sorted)
        loss= masked_loss(output,targets_sorted,inputs_sorted,4)
        loss.backward()
        
        output_arr=np.argmax(output.data.cpu().numpy(),-1)
        target_arr=targets_sorted.data.cpu().numpy()[:,:output_arr.shape[1]]
        mask=np.greater(inputs_sorted.data.cpu().numpy(),0)[:,:output_arr.shape[1]]
        
        accuracy_numerator.append((np.equal(output_arr,target_arr)*mask).sum())
        accuracy_denominator.append(mask.sum())
        

        model_optimizer.step()
        if mbs % 100 == 0:
            #列印訓練狀態
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                      "Step: {}...".format(mbs),
                      "Loss: {:.4f}...".format(loss.data.item()),
                      "Accuracy: {:.3%}...".format(sum(accuracy_numerator)/sum(accuracy_denominator)))
            torch.save(model, 'Models/word_segment_pytorch.lstm')
            accuracy_numerator=[]
            accuracy_denominator=[]
        if mbs % 500 == 0:
            learning_rate*=0.75
            test_features, test_labels,test_lens,test_ground_truths=get_next_minibatch(minibatch_size,False)
            inputs, targets,seq_len = torch.from_numpy(test_features.astype(np.int64)), torch.from_numpy(test_labels.astype(np.int64)), torch.from_numpy(test_lens.astype(np.int64))
            inputs, targets,seq_len = Variable(inputs), Variable(targets), Variable(seq_len).to(device)
            #所有輸入的sequence必須由長到短排序才可以輸入
            seq_len_sorted, sorted_idx = seq_len.sort(descending=True)
            inputs_sorted, targets_sorted,seq_len_sorted=inputs[sorted_idx].to(device), targets[sorted_idx].to(device),seq_len_sorted.to(device)
            output= model.forward(inputs_sorted,seq_len_sorted)
            sorted_idx_arr=sorted_idx.data.cpu().numpy()
            result=np.argmax(output.data.cpu().numpy(),-1)
            print('-----測試集驗證--------')
            for i in range(3):
                answer='|'.join(test_ground_truths[sorted_idx_arr[i]])+'|'
                pred=[]
                words=list(''.join(test_ground_truths[sorted_idx_arr[i]]))
                for j in range(len(words)):
                    word=words[j]
                    onehot=result[i][j]
                    if onehot>=2:
                        pred.append(word+'|')
                    else:
                        pred.append(word)

                pred=''.join(pred)  
                print('predict:'+pred)
                print('answer :'+answer)
            print('-----------------')
            
            
    
        mbs += 1
   
    

recovered!!
start epoch!
Epoch: 1/30... Step: 0... Loss: 0.6486... Accuracy: 70.877%...
-----測試集驗證--------
predict:然而|,|中策|組|不單是|特首|的|智囊|和|政治|顧問|,|它|也|是|協助|特區|政府|制訂|公共|政策|的|研究|機關|,|它要|經常|接觸|不同|界別|人士|,|吸收|不同|派別|的|社會精英和|專家|學者|的|意見|,|作為|特區|政府|制訂|政策|的|參考|,|這個|角色|和|協助|執政|黨派|與|反對|黨|派作|政治|鬥爭|是|明顯|有|衝突|的|。|
answer :然而|,|中策組|不單|是|特首|的|智囊|和|政治|顧問|,|它|也是|協助|特區|政府|制訂|公共|政策|的|研究|機關|,|它|要|經常|接觸|不同|界別|人士|,|吸收|不同|派別|的|社會|精英|和|專家|學者|的|意見|,|作為|特區|政府|制訂|政策|的|參考|,|這個|角色|和|協助|執政黨派|與|反對黨派|作|政治|鬥爭|是|明顯|有|衝突|的|。|
predict:由於誠|泰隊|的|日籍|投手|教練|中本|與|中華奧|運隊|的|日籍|投手|教練|酒井|光|次|郎有|連絡|,|商量|過|如何|幫林|英傑|進行|奧|運行|的|調整|,|因此|誠泰隊|將|盡量|讓林|英傑|不要|太|勞累|,|先發|的|投球|數|控制|在|10|0|球|之內|。|
answer :由於|誠泰隊|的|日籍|投手|教練|中本|與|中華|奧運隊|的|日籍|投手|教練|酒井光次郎|有|連絡|,|商量|過|如何|幫|林英傑|進行|奧運行|的|調整|,|因此|誠泰隊|將|盡量|讓|林英傑|不要|太|勞累|,|先發|的|投球數|控制|在|100|球|之內|。|
predict:也|歡迎|本籍|看護|、|家屬|及|助理|護士|、|居家|服務|員|等|相關|醫療|人員|報名|參加|。|
answer :也|歡迎|本籍|看護|、|家屬|及|助理|護士|、|居家|服務員|等|相關|醫療|人員|報名|參加|。|
-----------------


C:\Anaconda3\lib\site-packages\torch\serialization.py:250: UserWarning: Couldn't retrieve source code for container of type CharRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1/30... Step: 100... Loss: 0.6427... Accuracy: 71.785%...
